In [3]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [4]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codebert_embeddings/processed_embeddings/codebert_base_embed_line_by_line_avg.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-0.153420,0.293056,-0.017605,0.055680,0.082783,-0.264073,0.030169,0.041353,0.281950,-0.263212,...,0.552090,-0.065288,-0.011406,0.399021,-0.295382,-0.346146,0.336526,1,8077,critical
1,-0.134803,0.287359,0.010192,0.057800,0.042312,-0.167175,-0.049066,0.046448,0.154361,-0.159874,...,0.583667,-0.215274,0.060102,0.400974,-0.245126,-0.341638,0.333059,1,5553,critical
2,-0.140983,0.332473,-0.003147,0.057381,0.086165,-0.191188,-0.024117,0.044012,0.202778,-0.239297,...,-0.458214,0.535461,-0.120767,0.027277,0.383095,-0.258017,-0.341915,1,9341,critical
3,-0.156379,0.350339,-0.023524,0.044902,0.050281,-0.233743,-0.032882,0.050380,0.233215,-0.224153,...,0.447567,-0.113060,0.024070,0.371381,-0.312726,-0.301403,0.337536,1,10419,critical
4,-0.153420,0.293056,-0.017605,0.055680,0.082783,-0.264073,0.030169,0.041353,0.281950,-0.263212,...,0.552090,-0.065288,-0.011406,0.399021,-0.295382,-0.346146,0.336526,1,12232,critical


In [5]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [6]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8294
F1_macro: 0.7744
Accuracy: 0.8294
Precision_macro: 0.7839
Recall_macro: 0.7670

Bagging Metrics:
F1_micro: 0.8309
F1_macro: 0.7708
Accuracy: 0.8309
Precision_macro: 0.7898
Recall_macro: 0.7578

XGBoost Metrics:
F1_micro: 0.8306
F1_macro: 0.7765
Accuracy: 0.8306
Precision_macro: 0.7855
Recall_macro: 0.7697


In [7]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")



Final Test Results:

Random Forest Metrics:
F1_micro: 0.8179
F1_macro: 0.7626
Precision_macro: 0.7677
Recall_macro: 0.7581
Accuracy: 0.8179
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       814
           1       0.67      0.63      0.65       295

    accuracy                           0.82      1109
   macro avg       0.77      0.76      0.76      1109
weighted avg       0.81      0.82      0.82      1109


Bagging Metrics:
F1_micro: 0.8115
F1_macro: 0.7495
Precision_macro: 0.7606
Recall_macro: 0.7409
Accuracy: 0.8115
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       814
           1       0.66      0.59      0.62       295

    accuracy                           0.81      1109
   macro avg       0.76      0.74      0.75      1109
weighted avg       0.81      0.81      0.81      1109


XGBoost Metrics:
F1_micro: 0.8061
F1_macro: 

In [9]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/data_class/results/codebert/result_codebert_base_avg.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/data_class/results/codebert/result_codebert_base_avg.pkl
